## Acompanhamento Diário - Email 

Criação do email de acompanhamento

In [12]:
start = email_acompanhamento_diario()
start.iniciar()

Inicializando...



D:\Arquivos de Programas\Anaconda\lib\site-packages\openpyxl\worksheet\_reader.py:296: UserWarning: Failed to load a conditional formatting rule. It will be discarded. Cause: expected <class 'float'>
  warn(msg)
D:\Arquivos de Programas\Anaconda\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
D:\Arquivos de Programas\Anaconda\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


C:\Users\Fellipe\AppData\Local\Temp\tmp05hx0ibo
Carregando a 1ª tabela...
[]
Imagem 'Desempenho_equipes.jpg' criada com sucesso em 'C:\Users\Fellipe\AppData\Local\Temp\tmp05hx0ibo'!
Carregando a 2ª tabela...
['Desempenho_equipes.jpg']
Imagem 'Resultado_inspeções.jpg' criada com sucesso em 'C:\Users\Fellipe\AppData\Local\Temp\tmp05hx0ibo'!
Carregando a 3ª tabela...
['Desempenho_equipes.jpg', 'Resultado_inspeções.jpg']
Imagem 'Notas_prioritárias_pendentes_1.jpg' criada com sucesso em 'C:\Users\Fellipe\AppData\Local\Temp\tmp05hx0ibo'!
Carregando a 4ª tabela...
['Desempenho_equipes.jpg', 'Notas_prioritárias_pendentes_1.jpg', 'Resultado_inspeções.jpg']
Imagem 'Notas_prioritárias_pendentes_2.jpg' criada com sucesso em 'C:\Users\Fellipe\AppData\Local\Temp\tmp05hx0ibo'!
Carregando a 5ª tabela...
['Desempenho_equipes.jpg', 'Notas_prioritárias_pendentes_1.jpg', 'Notas_prioritárias_pendentes_2.jpg', 'Resultado_inspeções.jpg']
Imagem 'Fraudes_identificadas.jpg' criada com sucesso em 'C:\Users\Fell

com_error: (-2147352567, 'Exceção.', (4096, 'Microsoft Outlook', 'Não é possível criar a mensagem de email porque um arquivo de dados para o envio e o recebimento de mensagens não foi encontrado. Verifique suas configurações neste perfil do Microsoft Outlook. No Microsoft Windows, clique no botão Iniciar e no Painel de Controle. Clique em Contas de Usuário e em Email. Clique em Mostrar Perfis, selecione este perfil e clique em Propriedades. O servidor não está disponível. Contate o administrador se essa condição persistir.', None, 0, -2147221231), None)

In [11]:
# importing required modules
import win32com.client as win32
import pandas as pd
import os
import time
from datetime import datetime
from PIL import ImageGrab
from tempfile import TemporaryDirectory

class email_acompanhamento_diario:
    
    def iniciar(self): 
        self.extrair_tabelas_excel()
        self.enviar_email()
    
    def extrair_tabelas_excel (self):

        print("Inicializando...\n")

        # load files paths
        path_ws = r"C:\Users\Fellipe\Downloads\Acompanhamento dos Serviços de Recuperação de Energia - Email n.xlsm"

        # set lists
        self.emails = ''

        # close excel 
        os.system('TASKKILL /F /IM excel.exe')

        # acess excel file
        excel = win32.Dispatch('Excel.Application')
        excel.visible = True

        # read databases 
        df_files = pd.read_excel(path_ws, sheet_name='Tabelas do Email')
        df_emails = pd.read_excel(path_ws, sheet_name='Lista de Emails - PRINCIPAL')
        df_data_att = pd.read_excel(path_ws, sheet_name='ACOMPANHAMENTO DIÁRIO')

        # get last att date 
        data_att = df_data_att['Unnamed: 4'][2]
        self.data_att = datetime.strptime(f'{data_att}', '%Y-%m-%d %H:%M:%S').strftime('%d/%m/%Y')
        
        with TemporaryDirectory() as tempfile:
            print(tempfile)
            
            # algorithm loop - emails
            for j, email in enumerate(df_emails['CCO']):
                if j == 0:
                    self.emails = email
                else:
                    self.emails = self.emails + '; ' + email

            # algorithm loop - tables
            for i, tab in enumerate(range(len(df_files))):

                # flag loading 
                print(f'Carregando a { i+1 }ª tabela...')

                # set loop variables
                path = df_files['Planilha'][i]
#                 path_img = df_files['Local para Salvar'][i]
                path_img = tempfile
                sheet = df_files['Aba'][i]
                img_name = f"{ df_files['Nome da imagem'][i] }.jpg"
                init_line = df_files['Linha inicial'][i]
                init_col = df_files['Coluna Inicial'][i]
                end_line = df_files['Linha final'][i]
                end_col = df_files['Coluna final'][i]
                
                print( os.listdir(tempfile) )

                # open the sheet of workbook
                if i == 0:
                    wb = excel.Workbooks.Open(path)
                    ws = wb.Worksheets[sheet]
                elif df_files['Planilha'][i] != df_files['Planilha'][i-1]:
                    wb = excel.Workbooks.Open(path)
                    time.sleep(5)
                    ws = wb.Worksheets[sheet]
                elif df_files['Planilha'][i] == df_files['Planilha'][i-1]:
                    ws = wb.Worksheets[sheet]

                # generate and save the image
                ws.Range(ws.Cells( init_line, init_col ),ws.Cells( end_line, end_col )).Copy()  
                img = ImageGrab.grabclipboard()
                imgFile = os.path.join( path_img, img_name )
                img.save(imgFile)

                # Success Flag
                print(f"Imagem '{ img_name }' criada com sucesso em '{ path_img }'!")

            # close excel         
            os.system('TASKKILL /F /IM excel.exe')
            print("\nSucesso na extação de tabelas!")

            return data_att

    def enviar_email(self):

        def body(self):

            print("\nGerando email...")

            text = """

            <h2><span style="font-family: Arial, Helvetica, sans-serif; color: rgb(97, 189, 109);">Acompanhamento Di&aacute;rio dos Servi&ccedil;os de Recupera&ccedil;&atilde;o de Energia</span></h2>
            <p><span style="font-family: Arial, Helvetica, sans-serif;"><span style="color: rgb(63, 63, 63);">Prezados, segue o Acompanhamento dos Servi&ccedil;os de Recupera&ccedil;&atilde;o de Energia, o Acompanhamento de Pend&ecirc;ncias de Workflow e o dashboard de Recupera&ccedil;&atilde;o de Energia com dados at&eacute; {}.</span><br>&nbsp;<br><span style="color: rgb(63, 63, 63);">O dashboard, o arquivo de notas pendentes e workflow podem ser encontrados na pasta:&nbsp;</span><a href="https://iberdrola-my.sharepoint.com/:f:/r/personal/hfontes_neoenergia_com/Documents/Gest%C3%A3o%20Comercial_UTEP_UTD_EPS?csf=1&web=1&e=9wzfvS" rel="noopener noreferrer" target="_blank">📁 Gest&atilde;o Comercial_UTEP_UTD_EPS</a></span></p>
            <h3><span style="color: rgb(97, 189, 109); font-family: Arial, Helvetica, sans-serif;">Desempenho das Equipes Pr&oacute;prias&nbsp;</span></h3>
            <p><img src="local arq"></p>
            <h3><span style="font-family: Arial, Helvetica, sans-serif; color: rgb(97, 189, 109);">Resultado das &Uacute;ltimas Inspe&ccedil;&otilde;es</span></h3>
            <p><img src="local arq"></p>
            <h3><span style="font-family: Arial, Helvetica, sans-serif; color: rgb(97, 189, 109);">Notas Priorit&aacute;rias Pendentes</span></h3>
            <p><img src="local arq"></p>
            <h3><span style="font-family: Arial, Helvetica, sans-serif; color: rgb(97, 189, 109);">Pend&ecirc;ncias de Workflow</span></h3>
            <p><img src="local arq"></p>
            <h3><span style="font-family: Arial, Helvetica, sans-serif; color: rgb(97, 189, 109);">Processos pendentes de documenta&ccedil;&atilde;o no SAP KM</span></h3>
            <p><img src="local arq"></p>
            <h3><span style="font-family: Arial, Helvetica, sans-serif; color: rgb(97, 189, 109);">Fraudes Identificadas no dia {}</span></h3>
            <p><img src="local arq"></p>
            <p><br></p>

            """.format(self.data_att, self.data_att)

            return text

        outlook = win32.Dispatch('outlook.application')
        mail = outlook.CreateItem(0)

        mail.Subject = "Acompanhamento Diário dos Serviços de Recuperação de Energia"
        mail.HtmlBody = body(self)
        mail.To = ''
        mail.cc = self.emails

        print("Sucesso na criação do email!")
        mail.Display(True)

In [2]:
# Send an HTML email with an embedded image and a plain text message for
# email clients that don't want to display the HTML.

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
import smtplib

# Define these once; use them twice!
strFrom = 'fellipe.lucas@neoenergia.com'
strTo = ''
strCc = 'fellipellucas@gmail.com'

# Create the root message and fill in the from, to, and subject headers
msgRoot = MIMEMultipart('related')
msgRoot['Subject'] = 'test message'
msgRoot['From'] = strFrom
msgRoot['To'] = strTo
msgRoot['Cc'] = strCc
msgRoot.preamble = 'This is a multi-part message in MIME format.'

# Encapsulate the plain and HTML versions of the message body in an
# 'alternative' part, so message agents can decide which they want to display.
msgAlternative = MIMEMultipart('alternative')
msgRoot.attach(msgAlternative)

msgText = MIMEText('This is the alternative plain text message.')
msgAlternative.attach(msgText)

# We reference the image in the IMG SRC attribute by the ID we give it below
msgText = MIMEText('<b>Some <i>HTML</i> text</b> and an image.<br><img src="cid:image1"><br>Nifty!', 'html')
msgAlternative.attach(msgText)

# This example assumes the image is in the current directory
fp = open(r"C:\Users\Fellipe\Downloads\Tabelas_email\Desempenho_equipes.jpg", 'rb')
msgImage = MIMEImage(fp.read())
fp.close()

# Define the image's ID as referenced above
msgImage.add_header('Content-ID', '<image1>')
msgRoot.attach(msgImage)

# Send the email (this example assumes SMTP authentication is required)
smtp = smtplib.SMTP()
smtp.connect('smtp-mail.outlook.com')
smtp.login('fellipe.lucas@neoenergia.com', 'Cosern@01')
smtp.sendmail(strFrom, strTo, strCc, msgRoot.as_string())
smtp.quit()

TimeoutError: [WinError 10060] Uma tentativa de conexão falhou porque o componente conectado não respondeu
corretamente após um período de tempo ou a conexão estabelecida falhou
porque o host conectado não respondeu

In [18]:
def body():

    # site de formatar email: https://wordtohtml.net/site/index?tinyeditor

    text = r"""

    <h2><span style="font-family: Arial, Helvetica, sans-serif; color: rgb(97, 189, 109);">Acompanhamento Di&aacute;rio dos Servi&ccedil;os de Recupera&ccedil;&atilde;o de Energia</span></h2>
    <p><span style="font-family: Arial, Helvetica, sans-serif;"><span style="color: rgb(63, 63, 63);">Prezados, segue o Acompanhamento dos Servi&ccedil;os de Recupera&ccedil;&atilde;o de Energia, o Acompanhamento de Pend&ecirc;ncias de Workflow e o dashboard de Recupera&ccedil;&atilde;o de Energia com dados at&eacute; {}.</span><br>&nbsp;<br><span style="color: rgb(63, 63, 63);">O dashboard, o arquivo de notas pendentes e workflow podem ser encontrados na pasta:&nbsp;</span><a href="https://iberdrola-my.sharepoint.com/:f:/r/personal/hfontes_neoenergia_com/Documents/Gest%C3%A3o%20Comercial_UTEP_UTD_EPS?csf=1&web=1&e=9wzfvS" rel="noopener noreferrer" target="_blank">📁 Gest&atilde;o Comercial_UTEP_UTD_EPS</a></span></p>
    <h3><span style="color: rgb(97, 189, 109); font-family: Arial, Helvetica, sans-serif;">Desempenho das Equipes Pr&oacute;prias&nbsp;</span></h3>
    <p><img src="https://iberdrola.sharepoint.com/:i:/r/sites/Cosern_Perdas/Documentos%20Compartilhados/Dashboard%20-%20Gestao%20da%20Receita/imagens/email_acompanhamento_diario/Desempenho_das_equipes_proprias.png"></p>
    <h3><span style="font-family: Arial, Helvetica, sans-serif; color: rgb(97, 189, 109);">Resultado das &Uacute;ltimas Inspe&ccedil;&otilde;es</span></h3>
    <p><img src="C:\Users\B355502\IBERDROLA S.A\Cosern_Perdas - Documentos\Dashboard - Gestao da Receita\imagens\email_acompanhamento_diario\Resultado_das_ultimas_inspecoes.png"></p>
    <h3><span style="font-family: Arial, Helvetica, sans-serif; color: rgb(97, 189, 109);">Notas Priorit&aacute;rias Pendentes</span></h3>
    <p><img src="C:\Users\B355502\IBERDROLA S.A\Cosern_Perdas - Documentos\Dashboard - Gestao da Receita\imagens\email_acompanhamento_diario\Notas_prioritarias_pendentes_1.png"></p>
    <p><img src="C:\Users\B355502\IBERDROLA S.A\Cosern_Perdas - Documentos\Dashboard - Gestao da Receita\imagens\email_acompanhamento_diario\Notas_prioritarias_pendentes_2.png"></p>
    <h3><span style="font-family: Arial, Helvetica, sans-serif; color: rgb(97, 189, 109);">Pend&ecirc;ncias de Workflow</span></h3>
    <p><img src="C:\Users\B355502\IBERDROLA S.A\Cosern_Perdas - Documentos\Dashboard - Gestao da Receita\imagens\email_acompanhamento_diario\Pendências_de _workflow_1.png">   <img src="C:\Users\B355502\IBERDROLA S.A\Cosern_Perdas - Documentos\Dashboard - Gestao da Receita\imagens\email_acompanhamento_diario\Pendências_de _workflow_2.png"></p>
    <h3><span style="font-family: Arial, Helvetica, sans-serif; color: rgb(97, 189, 109);">Fraudes Identificadas no dia {}</span></h3>
    <p><img src="C:\Users\B355502\IBERDROLA S.A\Cosern_Perdas - Documentos\Dashboard - Gestao da Receita\imagens\email_acompanhamento_diario\Fraudes_identificadas.png"></p>
    <p><br></p>
    <p><strong><span style="font-size: 8.0pt; font-family: 'Arial',sans-serif; color: #5c881a;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Atenciosamente,</span></strong></p>
    <p>&nbsp; &nbsp;<img src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAMCAgMCAgMDAwMEAwMEBQgFBQQEBQoHBwYIDAoMDAsKCwsNDhIQDQ4RDgsLEBYQERMUFRUVDA8XGBYUGBIUFRT/2wBDAQMEBAUEBQkFBQkUDQsNFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBT/wAARCAB1AL8DASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD9U6KKKACiiigBvFOrNs9Xt7y+vLOI5ntGVZl/u7kDj+daOayhOM/hAWiqlrfQ3gkaJ9/lyNC3+8p5q1mtQG7QKXFV7G+h1K3SeBxJE4yrDvVg1nCcakeeAfCLS0UVoAUUUUAFFFFABRRRQAUUUUAFFFFABRRRQAUUUUANFYXi7xXpvg3QbrVdUm8m2gTPYszYOFUZ5Y1zfxO+LWi/DCw3Xj/atQlX9zYQf6x//iV9zXyB4/8AibrXxI1yCXVJv3G/MVjD/qYk/wA/x18bnXEFHLKcow96ZPMe5/s//EC58SeN/ENzeDy31p3uY4f+eezhE/74r3DxV4lt/DOmNdTEFyMRRf329K+Q/hnqk3hvxNpGoQbfs9s/l/P/AB/JsrtfFPiy41i4e4upt7/+gV1cCYbGYvAOtjY2jKUpR/vRZw43HQpe5D4j0/4OeIW1KTWbWZvMm8/7T/339/8AWuo8eeKovDelsI3/ANMnUrEmf/Hq+f8AwL4suvDWuTXkJXE1u0Wx/Y5D1Y1rWp9QunubmZppG+87cV8R4g8c4bLp1ctyuXPVf2vsx/8Atj63hvJ62Kpwq4mHLH/0o9h+Duq/btDltt3z2kpX/gLfOP516JXhfwF1QSa3qloWyXiWX/vlv/s69zr6HgfGTxmQ0HPePu/cRnFH6vjZwQ6loor748UKKKKACiiigAooooAKKKKACiiigAooooAYq8Vz3jnxVF4K8Karrc48yOxt3m2d2YfdX88D8a6EV5z+0Bod54i+EPiexsFaW6+zedGifxlHD7f/AByuLFynCjOVP4gkfDPiXxZfeKNQvNW1G4aa8uJsSsOn/wCxUmhyJsgubj7+z/xyuN0q++1XPk/6yGZPnrR1jWPIh3p/BX4lleGpY/OsPh8X8Mpx5v6/vHi4jETo4edWB38HibzNSskR/wDlsldRdax583+xXhXhnWHutY87/lnbfP8A8Drs9M8RpPfPDv8An2fJX9P8cU6+A4exU8D8UY//ALX/AJKeFwhUo4/OcPSxPwSl/X/kx6NpWqeZqifP/frQutSri9FuPnmm/wCAJVy61Kv85KtByqH9mywyUtNj2H9nmR5vHF638C2bD/x9K+kwp5rwz9mPwzLDo+qa5cJsN5IIrfjPyJ/F+LZ/75r3RmO2v654IwcsFktGM/te995+G8R1YVMxqOH2R9LSUtfenzYUUUUAFFFFABRRRQAUUUUAFFFFABRRRQAUnWlooA+QPi9+zjpeqfFizsvCc0WkahrVpdX9zBKp+zIEdPnQJ9wu71i6b+w34pvLxF1fxHptrabvmaxjkmfb6fPsr3PxRZ6pD+0p4N1CKwup9KfRb2zuLqOBnhicujpvf+DOyvUNa8QaZ4ftUuNV1K10yFm2rNdSrCm70yxr5Cnk2CqVqtecdeY5XRhPc+VPF/7C72sax+CPECwQ9XtdYBfL/wB/en/xFY3g39hfxFLrltc+JPFFraWUL72j0eN2mk/4HJ9z/wAfr7MtLyC8t4riGVJ4Zl3JIjblZfUVRh8TaTc6nc6ZFqdm2oWy75rVLhDNEvqyA5FfojznFOh7Cc9JHkxyTBU60a9OPK0eEeIv2WrxJH/4R7Vbf7L/AAQ32/cn/A0o8L/sszm+jn8R6lHJar/y62W8bvq9e0f8LR8Gj/mbNE/8GMP/AMVWjo3ifR/EUcr6XqtlqaR/K7WVws232+WvyyPCOR/WPbRhr/iP0X/WbM/Y+x9toXbGxg0yzhtraJILeJdiInCqoq5WBqXjjQNFuBbX+t6dZXP/ADyuruON/wAia2YLmO6hSWJ1kiblXVs5r7iHs4rkh0Pmubn1JdtG2uWb4n+D1ZlbxTooZf8AqIwj/wBnqe18d+HL60u7q317TZrWzXfcTR3iOsS+r8/J+NHtYfzC5jpKKzdK1ew1uzW7069t9QtpOk9rMsiH/gS1BdeJNKsNUg0yfU7ODUrgfubOS5RZn/3EPJp88N7jNfbRtrmbn4j+FbO5lt7nxJpFvcRtseCS+iR1b0ILcVJY+O/Duref9i17Tbr7PF583kXaPsj/AL7YPC+9L2sP5hcx0eRS1yo+KHg3/obND/8ABlD/APFV0FvcQ3kKTW8qzQuNyuj7lanCcJ/CxlqisTWfFmi+HyiaprGn6a8n3Fu7lId//fRqS38Rabdac+pQ6hbT6eisz3UUyvCm373zj/PFHPDuBr0VmaNrmna9Zi702/ttTtidontJklT/AL6Wq954s0XT9Sg0261aytr6faIrWa4RZm3cLtQ8mjnh3A26KKK1AKKKKAG/xV80/t5f8kn0nd/0GI//AERPX0t3r5q/bsgmuPhTpqwwyzv/AGpH8kab/wDllLXk5n/udRGNX4Dm/gP4+1T4L+KrX4Z+NJt2kXkSXOh6m/3Du/5Zj0X/ANBY4703wQuf2vPiqfXS7j/0C1r1T4m/Bi0+MPwp0zT5ClrrNpaRTafeN1im2AbW/wBhuhrwD9myDxGfjV4xfxLDdprSaFcQ3Ml0md7oYE+//H9yvna0a2HdLDz+Hm93/wCROeXNHkgc7+zxqHwasvCF4nxHi059Xe8L2/2u3d5PJ2J/cB/j316r46+JXgj4ffA3WtY+EH2WybUNQTTpbyxidNk+wvu+cfeCf+hiuI/Zj+KnhL4a+CdQsPFel3s17NfmaIpo8k3yeWiff2f7Feq+NIvD37UHwp1rR/BkM1je6bcRXMMV3am0R5trfL9GQuM+4rLBp/VeSlKPPy/9vEx+D3CX4bfsk+CJPBlnd+J7CTxDrmpW6XN1d3VzISsjruOz5uOa5n4W/avgX+0dP8NIb64vvCuqW5ubOG6febV9juvP/bN0/wC+Kv8Ag39qSXwP4cs/D/jfwb4itfEGnwpbf6NZeYlwFGFcHPFS/CLwj4k+Jnxou/ir4j0mXw9YQQeRpWm3f+uOU2bz7bHf/gTn0rrXsJOl9Vj7/wDXNzF+57nKeLfBC8+Eun3vi0fEuLTmme//ANBN1C7nZ8+/7gr2HxDP8K7z4CfE2X4awWKItiqXz2kLpnrszvHpvrzz9nf4jeF/hrceMYfFml30z3l+XtvL0qS5+RN+/wDgr1fx18UfCnxI+CfxEtPC9jdQz22no8qSac9vv3t8mOPmrkwiX1R3lHn5Zf4iKfwHjvwI8cax8AbXw/rGpFp/h/4qL73XpZzo7pv/AN/5P+Bp/wBc69D+KlxFfftgfDK5tyJY5rON0mQ5VkzPXZfBL4f6b8QP2W9E8Pa7bMLW6imB3JseJvtEmx19GrwDwb4L8UeB/wBo7wb4f1/7RdR6TOYbG6ETmF7bDumxu6/P0/gqeWvhsPSh9mfL/wBuh78YQLWg3Xw3s/j98S3+I8Vi2nvfzfZPtcLv++875/uCvXdFu/g1qHhLx43w4t9OTVo9AuhcPZ27o/kuhP8AGP72K858F+NtB+HH7QHxOvvFGnXc9leXkyW5j057j5/O+levS/GLwX4+8J+M9J8NWF5a6h/YV5M/m6a9tvRY8ffx6uPzrfBpezn70ftf4gjseI/B3Tfg7Y/Be3v/AIgWmnjVrya8S3kKP9pmRDzs2c10vwA8T+Ivhf8As5+NPEV1b3C2FtKX0WO6T+N/k3qP+ee90/J66P8AZx+GOkfET9m06PrunB3a5vI4pp4QJbdycb0/uVm/Bf8Atj4h/D/xp8GvF0lzDq+mxPDZ3dyr8xo+E+f+MI6of9xhWdCnUh7OUfd5o+7/AIv7w4xfuD/gz8CfCvir4fD4h/EmX+3tR1VHvJrnVLp1ht4d59xj8a9BjsvAGg/Bvx5ongLUNOuLKPTby8mtbG7Fz5ZeA5P3z12eteZeAvixffBrwe3w++JngnVLqytFeGCa2svtVvcw7+E/uv1rJ+Evhi+ks/jT4mtfDl5oXhzVNJul0yOe28n5MSNsRPxrop1YQhGFOPv8vvfzFx5T1P8AYneO1+A8Ts6pGl9dO7Z+6MivnvxLoerfG6T4kfFTT5pYk0W8h/szy0+/DDz/AOOJsk/4HXR+GfGV54F/Y3urS3huE1PWNQudPiRIn3ojn53/AO+P/Q67jwD+zn8S9E8E2um2HxBt9D0+6iM0ulnTI32SSp+8Rn/j5rD95iqVKhCHNyx//ZM/ihCB7t8H/iJD8Tvh3oviGIr51xDtuEX+CZTskT8HBruFXr718mfsxtqnwg+KHiP4YayJJbWXF5Y3XlMIWfbzt/3k/wDRdfWgPzYr6fAVpYigvafH9o7acuaA+iiivULEpOPTNHauT+InxC0v4a+HX1fVRNIm9ILe1tk3zXEzH5I0TuxrKcowjzTA6vd9KTI9BXnnhbxt4y1y/gGq+AJtD06dc/aX1SGV4vZ4gP5Vg2/xy1rxRcX7+C/A9z4n0iyna2fUmv4bSOd1+95O/wC+PfiuP63R6/8ApLFzHsW1fQUceleP3H7R+jnwHb6/a6bqFzf3Go/2Mmh7Nlz9v/54Pn7h4rUi+JXiPTfD2uat4n8Gvo8emWUl6gh1GK4WfYCdgx91qpYujL4JE80T0v5fQUcdgK8o8MfFTxf4pt9JvofhxcRaXqCQzrdyatbnZE/O/Z97Ow5rLi+P2t6pN4kk0fwBfarp+g309jcXcV/CjuYvv7I3wxqfrlJf/ssOeJ7XtHXFG0AYrxPxp+05onhXwR4X8WWul3eqaZrsmyPy/keFVyZCyn+6Fb8q7TxF8UNP0hfCEttEdSs/El/DZW88LfIu+N5Ef6fJR9aw0n8X9SHzRO76UbR6UtFehZFCbR6D8qNo9KWiiy7AJRgelLRQAm0elGB6UtFFkAm0elFLRQAmB6UtFFMAooooAb2rw39ov/iT658OvFF5u/4R7Rda8zUWz+7hR02JM/sj4/Ovc6qXllBqFvJbXMSTQSrseGVdyutctel7anykyjzGNqGvaZqGliKHWLON7+NorWRbhDvdh8uz+9+FeW/sv+INO0v4TW2hX9xHYav4flubXU7W4kCNDIJnbc/1/vfWvQNC+D3gvwvqKX2k+FdJsb1G3LPDZoHT/cP8H4VP4j+E/g3xdqCX+s+GdN1O9TpNc2yu/wCZrllSr80anu8we8eCeBvCOgfFCDx1d6zemz0bxB4me80G6jufs8zvCmzz4T1pdS8TeItItfiH4A1vXLfxdaW3he6v4dWVNk0WMx+Tc7P4+9fQ2t+AvDniPSodL1TRNPvtOh/1VrPaoyRcfwDHy/hUGi/DXwt4b0m60vSvD+nWWn3a4uIIbZQkvGPnH8X41xvLpR+Az9meV/BnS54fDfhK6k+JlzNCbG2b+xX+x7D+7H7r7m/9c8V5St54q0PRfiNq+kapcJ4eh8X30OuWVjCn2pLXjzJoHf8Aj6V9NWvwY8B2N5FdW3hHRIrqGRZkmjsYw6P/AHs4roLPwzpWlrfLaabawJfyvPdokKjznf77v/ezUvL6s4Qip8vL6i9keEeKNG8Nxp8DbDQPLu/DL6i8dsobzElha1k+969a4RY734efEbwb8Mb4SzWen+KYdT0K7f8AjsWjceT/ANs3f9K+p9P8B+HNKh0+K10Wxgh02V5rRUgULbO/3mQfw9+lWNV8J6LrmqadqN/pdrd6hpr+ZaXM0QaSFvVGPIpyy9z/AHi92Xu/oP2RujoKWkpa+hNgooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKAP/2Q==" /></p>
    <p style="line-height: 110%;"><strong><span style="font-size: 8.0pt; line-height: 110%; font-family: 'Arial',sans-serif; color: #5c881a;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Fellipe Lucas Filgueira de Oliveira <br />&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span></strong><strong><span style="font-size: 8.0pt; line-height: 110%; font-family: 'Arial',sans-serif; color: #5c881a; border: none windowtext 1.0pt; padding: 0cm; background: white;">Unidade de Recupera&ccedil;&atilde;o da Energia</span></strong><strong><span style="font-size: 8.0pt; line-height: 110%; font-family: 'Arial',sans-serif; color: #5c881a;"><br />&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </span></strong><strong><span style="font-size: 8.0pt; line-height: 110%; font-family: 'Arial',sans-serif; color: #5c881a; border: none windowtext 1.0pt; padding: 0cm; background: white;">Rua Mermoz, 150. Cidade Alta, Natal - RN</span></strong></p>
    <p style="line-height: 110%;"><strong><span style="font-size: 8.0pt; line-height: 110%; font-family: 'Arial',sans-serif; color: #5c881a;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Telefone Fixo: </span></strong><strong><span style="font-size: 8.0pt; line-height: 110%; font-family: 'Arial',sans-serif; color: #5c881a;">+ 55 (</span></strong><strong><span style="font-size: 8.0pt; line-height: 110%; font-family: 'Arial',sans-serif; color: #5c881a; background: white;">84) 3215-6492</span></strong><strong><span style="font-size: 8.0pt; line-height: 110%; font-family: 'Arial',sans-serif; color: #5c881a;"><br />&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; E-mail: </span></strong><strong><span style="font-size: 11.0pt; line-height: 110%;"><a href="mailto:fellipe.lucas@neoenergia.com"><span style="font-size: 8.0pt; line-height: 110%; font-family: 'Arial',sans-serif;">fellipe.lucas@neoenergia.com</span></a></span></strong></p>
    <p style="background: white;"><strong><span style="font-size: 8.0pt; font-family: 'Arial',sans-serif; color: #5c881a;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Whatsapp de den&uacute;ncia&nbsp;para recupera&ccedil;&atilde;o de energia:&nbsp;</span></strong><strong><span style="font-size: 9.0pt; font-family: 'Arial',sans-serif; color: #5c881a;">99431-6873</span></strong></p>
    <p style="line-height: 110%;"><strong><span style="font-size: 8.0pt; line-height: 110%; font-family: 'Arial',sans-serif;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </span></strong><strong><span style="font-size: 8.0pt; line-height: 110%; font-family: 'Arial',sans-serif; color: #5c881a;">Uma empresa do Grupo Neoenergia.</span></strong></p>
    <p><strong><span style="font-size: 8.0pt; font-family: 'Arial',sans-serif; color: #5c881a;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Siga nossos perfis nas redes sociais:</span></strong><strong><span style="font-size: 8.0pt; font-family: 'Arial',sans-serif; color: #5c881a;">&nbsp;</span></strong></p>
    <p style="margin-bottom: 5.0pt;"><strong><span style="font-size: 8.0pt; font-family: 'Arial',sans-serif; color: #5c881a;">&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;<img src="..." /></span></strong></p>
    <p style="margin-bottom: 5.0pt;"><strong><span style="font-size: 8.0pt; font-family: 'Arial',sans-serif; color: #5c881a;">&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;<img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAACYAAAAmCAYAAACoPemuAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAADsQAAA7EAZUrDhsAAAn7SURBVFhHtVgJVJTXFR6zHIEBREe22XBBQ61pEj2mtgkmxmo0YkFZBph9THFN1aIGTezAMMAwK4s7SKqA4hI0sVFwi1UR3K3irhHEGJG6IKKAyO19bx4BFESjfue8M3P+ecs3d/nufT/nRTBVO8B5YkJvz0lJPr1kCcKBsqRe7yjNPn5Sg69QnujHG/2Fb1c29dVDE9/LR20RjlenCA1SY4+CED33wvg4h9uBWoe6QG3XhsDYrveD4hwqwxJdjimsnrlqi2CmMkH8QajW3Zlt8XKh0fHf0lhF89UpnkXB8VwIMzqAwuYB05f8HmK+8YfYnDGgyx0HC7JHQnTmEIha2Buklm4QYugKEkO3qokpgrUKkyAkQOvtxLZ8MchMnlylWThNZfM6Hm7kgtzaHWKyPoJV2/8JJac3w4WrR+D6rZ/gds11uHOvCv5X/TNU3DgDpZf3wrbDWZCycSJMWeQHEWYuRJrcalVWfr7aIPqQbf/bILP09VWZ+RsizW4NUrMbWiUA9p7cAFV3rsKzoq6hFsqul8KaXXqYtmgAhCNBpdX9qsosmhG1bPCb7Khnh8IgfF9t4x8MN3FhysIB8EPJEqitq2bHPT+ampqg/PoptODnaLnuEGFya1BZhamaOTwXdmTnCIvrPQRddzIs2QG++mYUnKs4xLZ/cTxsrIfNJYtAaRGge11AbRUvRct1HneSeFFfpcXrCCGlywmCytvlbMuXi32l34LaIkbLdQOlUZyo1XJeYxSeRCjqkgJjisTB/Ky/YCxVsG1eDXYfXwNysye6lvdAmiyMYDSehCJROD3S1K1xcrofXLp2nC1vQc39W+jWg3DmSjGcrSj5dZyrOABnyovhxE+74VRZUavfDtgHmX+lBG7W/MJ2agFJinCTMyaE9wkpShKj0gJJIr+/wuZ1IhK1Z+vB5WxZC06XF8GXGR+DwigCtbk3aMx92g5Tb+ISUJl6/fpMmSwCaZI3Di/6OXPJUDh4bgvb0Q7yZ7XZY5EcZqtRbEAqXeyM7OiiNItiJKhTcSgJ9+tq2DI7Hj1qhKS8MPhsPgdC47tDmL5HO6PtczIvOM4VJHoehCf0hAlxzjAG15Nkejy7D58rBIXVA6Qm3mVZkuAdxonDCdELBXKzV7HU0p3q1OOofXAH5iwfhge5gMzA72RgtiV6QCRaKaswBo5eKMSxDWzfToSAr9+AKWkDoaLqLNvZDmKI5LUREIoJp7b4RDNaRN1F40ISufVfZvnDrbtPxgEhNi/zE2qB9sk0DwEOIVrLjbr7fCuZ2XtiPQQseAMmpfwOyipPsqct2P3fPJChYWRG912yaE8uJaa2iuJDjY6wctt86rbH0RmxyCRPCNF1Ayzk1KrEbRpLXzh/tYXYj8dz0GKvw+QOiBErzlg6CMbruDV/M/r25QRoBztJk3t+L7fxoKg0n01ri46JCWgMEVKzl32A7lLDgpVj7PGlc4X0TZMwNNbBnhPrqKvGLXiTurKsspTt3IK6hvtgWi+FYD1pDgRjOBHa/j1D9C6nJi/qR1O7PTQTIwQkGNjkkxAjQU0ybsMeI9y+dx0aHtZhvNyFwsMraIYGaR3pHPI5PpaL61xhatof2iVGkLszFsIMXJAaPWZyZun7CYLiHCv/kTHkiaBshp3YcNzcCWYsHgLzVoygwU1clvejns1qix9KFtPfw9B6czP8YfZyf0puatrbHRLbcnAZRBrdIMLAs3I+N4l7B8V2rZ73r487LD+1D6px82Ew9qsukL/PSgWWECOaVVXdfnW4dfcafLFoEA34Yxe30xI0FmNsUuqADontPLoSZCYeSBLdMjDw+/VBYnfnI7EbTyOGcvEZEttcvBCuVp0HBYonybBadF17uHPvBpUYsqa0bC8cOLvZTizlacRWUWJhiW6ZdlfGOlbNzvwjHniOTWmL1sTy91roodPT3wVcB8cu7WCz2uL0lf1UaIllL/58FJNgPVr8NczKjokVHMpAV3YHSRIvlaNO8HUP1jufnbr4LUzvw2xKW9hd+REqdxdYgzFFeivLejmMmMuBGIy9K5Wn2Ew7iHvjsTMZPpsDuuwgqvQ7j63qlFjebj0GvzPIkzzn0NY5IplXqMT+/cCZf7MpbdFCjAM5u+LosyPnCyEy0QtGzOFQ623anwonL+9BvcqF+ZgcI5F0eII7HGK1sfBw5lOJNWCfZstXoVw4gtwoCiT62kVt8zZJUGBX74pHazxiU1vQ2pXEYs3YeSybHjQqhkOtNw4D/VP8/gmSJcWekGlGQSfErt28BNEZQzE8nOqUCT5+VPkVJnEIMm1asHIUvVA8jmZiJCvX70lmT+0g/Xx+kQ0SVodA9NI/Yx83EjK2RGPm7mcz7NiOGTf269fswX/9SeUvPv0dFnJ3iDDyiqVaX1dKjHStMpP7CaXNHSt9AZvagnuoY3OWfUgtloMiSPCoqREaH9XT7wT1qNw3USLu1t5kT7CNftRA5xF8X5xO1xNi5fhnWoOsTdsYBaRr1ljFWg6w1oe0tUqTMEGCPZFxXSSd2Br1Dx9QUSXpPhl1iFhHlxMIsasC6CDfk/IktOwY8sJBvzoYYrPHgZb8lh0ICbnBMC39bVzfBbVtMNx4rDM+Xb4fNCkiFNYeN6R6n6GUVDPCdfz3FBb3S3JLz3ZbHyKswXFuEIBx8uk8DozGRGgzyLPWo9VzMp9Ya0KsM+Ts0NKsbga53hnXSfFa5wQqs3AJJ5TzOqPUAryyaSNMrk2zlr2Pl9gyttSOhsY6mnFZBXMhY2s0ZG6d88yDxNwK/NyBcfag/h7b0Y4tB5ZCpBlrr9mjPFzn8x6j0hYhqGkKi/fOcJMTukTSJl5eBY5f3AUaWy/Ay3SDzCD+O6PRPpQ68WC52eNyWLIjpGJA1tx/NeROYZmakj4Qr2/O6EJB5jO901CaxGPlFo9fwpKdwIT+76iG/lYUncrHJPKzx5VVsFEew+exozuHwiz6K14QLkvQrXMz/bEqbKb91ovgdk0l9lxavKrx0VIu6EbB2lCtjxc78tlB3mupU/h7Ik2uoLR5QdqmKCi9vO+532GQi/OOI6voWyLyWkBu4VVjBlqVWh83dtTzQ5Ms4qvMIrPC5l5BrDcxTYzpHQEFBzOp/lTeLqOV4WEjiineF+obHtDqUY6FnYh17s44iFnhD1KLG4QbXfAPehcpTPwwtv2LQ5os8lfZxEukxh5loUkOEGlxhUnpvmiFYSiuIfQ9WPp3k8CyQUHvprMyBoEq1RtIDQ7Rc5vUqd7/wUvPLHJVZFu+RGCFkKPWaCw+U+VWr9Whia6ng+IcaoLiusKEeAc6xuscyG2pMVjnVBmR7LZPnSKwaDBepQaBkO3yakFe/EaZ+/ckVy1NIv9PKoNopMwoHK2wCIYrjT7vkgY0ytCnG5v+nOBw/g/sbf3nVKpMFwAAAABJRU5ErkJggg==" /><img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAACYAAAAmCAYAAACoPemuAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAADsQAAA7EAZUrDhsAAApOSURBVFhHtVgJVJRHEh4TV05FQEBgAM31EmPcxN012Rc3m+yuyXpzDcfAXKAmumq8YozXcMwwJzOIB0JiFON9xlW8V41LXE9ijAYUT0RUFBVREVC+re5pA2M08SAfr9/76b+76vurqquqR/Kk0Golz6i0Ye0H614OTM58/gWlvnM3NtgzmxtifM6LrRHLf3O0Upo7d0uyhqmV9sCcOKNXYVSae1l4ikvNQK1rPR/0HKVzL4s3e+3U2ANnsrVyg/Q1sb/loTRI39HYQmYmWn2PRWe4Ic7qieTsYHyc+zomznsPKQv68jFx3rsYmft7JNG7OKsHomhtorVDicYekq0wBvYU4p4eKkNAJ5U1aLrC1qE81uyO5GlS6BdH4etvs/Dd8a04df4QKq+W4dqNSj7Y80maKyrdglX/tSN9YQQ0WVKwvQq7X5nCLM0amv58iBD/ZFCbgvuoswL3xVs8obEHY8aaofjh5E5cv1WFR8X1m1X4/sQOZK8eDHVWEJgskrlbaQ55X6h5LLRSmUKHkAsux5k9MDm/Fw4c24T6httC3eOD7d1Xsh4T5/6drOcBZabfRY1RqhH6Hg1qi3SY3OxTG2tsi5x/DycXXRLinx5Xay5yy8caPUE6bihNoYOE2l+GwhgWSZaqjjO2w4L/aFHXUCtEthxu199E/uZJRK4tOxhXFNbQfkL9g5Gol3ZVWANK48yemLNhPOrvPLnrfg3sg3MLRnG3KmwBxQnajl0EDWdEj5a6qa3Bi+PpiGcsisKt29eFiAej+uZlXKo+i7uNd8SMA+z/81XHcfeu83xj411culbGD8M93Ki9hrQFAxCfSTFnDZovk0naCDpNUBhCZfEm77oPp7+IkxUHxdaHY29JAZZ9Y0AtuaU52In9ctOnRNw5LhnRVYWZ2HVklZhxoLR8PwZnPwe5yfsWeSxc0HFA9qm3l9oWvCbO4o6lOzLElofjzt0GfLF+LMblve1kAYZdR1ZDZeqEI2e+FTMOVFOOGzHjDczdNEHMNGHB1lTEkm61TbpCpvXzFLQkEo0h9B8xGe1uDM95DRWXS8XyB6OR/r47vgW56z7GR1mvovTcfvEGuHK9Ap/kvYPek1ph3e5ZYtZhrfzNE/HBZxLoF0Wj4U69eONA2cViDJvVhQ6D17UEQ9i7ghblrEypXmZ2ISuM+9mm5rhLcbLt4AJsP7gQmw/MRVSaDwp2z+bvzl0+htSvBmDgVBeEa90wjRIqQ139LSzepkOMzgcDprTByBndaa3zxzfcqcNs+tAYkwuSbKFaxkmSoH2hXbzRd6fK7o89xWvF0p+jqvocCvbMxsZ9X6C4bBcn0WfSs3yuiCw4ZvafiZQr4jP80H/K75C1KpkOwUlOMCLFA7E6X8Ske4MyPg6UbhZSm7Dr8NdQ2jog0dxhK3enwvhcaESqe9Wo3DdQVvmjWOaMI6cLKael8MBlbtm4/wtEpbZFXsEYzNn4CdSWMFLuTqT8yTLeMC6JpYNhxNjct7kF4/QdoDCFIFZP5Ijg1qL5QnITTl34ASNzupEXPM8rdS8GSxIzOr1JbQvSFw1EzX01kJHYW7IOi7al4Vj5Ppy5eBiVdORNS+U8xqavHkKW8IEsrT0SjB0RneYF20o1uS4dg2wvcbIJGR2JVDASjVQniTj7oDW7soWGJrAGIOWrPhiodWlUpYX+QaKxBvWL1Lli+teDiUiDWObAoVPfYP6WyaiioGbY+l0+DItjkbduFLlqEKJJSZyOWUOKiFRPTJ3XG0u268iCnThJRqb5kGcEIDLFk1vzfjQ01MG+So3IdBeozB3/KVGYA+XRGe4U+GPEEgeu3rhIJ2sm9pSsxZWaC3xua9E8DMl6BflbJlFKCOUxw6zBXKWi2FlI7h5PpzKSSN5PqjmxFTstXN79mL1uBKL1blCYO8okiVb/BAexceK1A8fK92LHwUWooCy+YV8epYILMC9LgG5hFKwrFNxNjBQbjMj4z9/BXEoJKnMYD3SF0eG+pkEfkNGBLNke6/fmCS3OYOERrSf3WwJiJGpz4MAonRtV/I94OrgH5sadh5Zy9365cTzsKzWQGwJ4Z5C2YCDCU+jLGDFSGJnqAW1+X/rikUgwdOSx5EwqiK+Tpbfnz/uObhBamsDS1LTVSYjUuUCdGdxHotKHvh2e6ooM6kpvUu26h+Ky//2UJAsPr+Axw459NgU8s1y41pUru2exCXPe46WIxVssufZBxOhOQNWiJ4XGeS63OWqogqQvGkByXaA0SntIBlle6hyR4lY99vMeKKckeQ8s4Jds16P4zC4cOrnjJ8Gfzfkbb4eYZRghpVlKLvKDxtqZp5RxlCL6T23jIEMkHe6W0sn14vubV4TmKKssxui87nSI3KtYCpOwa1ac0XuvJiuQulTnxFdydg/VTgN1G9GUNFuj3+TW6DOxNazLVZi5Zhgi09qi35RnuSvfnyChWtidDkAqPrS/jL6TnyEibjwW+9O+AZTPZq8dgdq6G0K6M1h3q84KQLzRp5AlfQm1Gs+qbUH2GLMbFdMUpzhjKD13AJO+7IXROW/h45w/YVROT8prOt7ybCnKR8r8/rzMjJrVA5blCpy9VIIfqYDbVyXRnjcxnAr31Pw+WL8nl0jVCKnOYE3BPDo4jEOSPcT0031UYQrtF633rB/7+VuksFwsd+DStbNYQh0Ha0/KLx/9WQ9Wc+sKJ3O+6oRTt8vqH5s7Sy5ivdsv4eLV0xiV2x2yDM9apTGsNyfFEKH18FfYArexu+La+2LgKLmTtSrXf0X404BdBdkdVJUZtCl8QltfQcsBlVGaHGf2ujMi53VefO+hpGw3zEvjKZ+dEDMti3LqNP41qyvo5t6QaAhRCjpNoNbaR2ULKmDMWUfAXMHA3MGKdcWVlifGXM9qK7eWLXANa1gFHWfIKaclWvwr5WYvukVn8s2NjY28S/2lPu1JsWyHkSzVDgqrf0ViavCbgsaDQf3SULnFuyHB7IcNDykdLYG1VIflJl8qPz51Kos0Wah/OP6qlbRWZkp15HMwcst3mHgX2lKorbtJp1xPsn0Rb2rPErSW8Gg/V8m0XdpQt5BOlrvNCFpXKHH6wmEh+glBIXGy4ntYqJzFkfvkVp9astRUZgih9pHRSmUNHaK0+Z9hwTlsZhdeBc5WHn2seGOHiOUy1vcPnfEy4i10wc30P0X18PF+t7gfg4xhb2lsQSvpQNyKJaEj6CbFLg6Fh1fi+LkiXlNZmWHZmw32fJnuB8fLi1D4w0rkrB2O4TmvIpY+Tm5pf1NjC16WZAr5oxD/dBiiDXSnVkSWNE26MtbQtkZmos7C7ssVTp7fi0pRPLLXDMZ0GmZ6npTfC8NnvQolrWFrYwztqpOygpar7cGRKm2YqxDbcpBRwVeawt7T2KRTEqy+BdE6z9PhKa63I1JdqCV25SOcnukOcTs63eOUwuq3Tp0tnayyhL2boPVpJ8T8tki0BHgkZ3UNSLaHvTLIFPIXhVH6AR/WkJ5sLlnfOSBxbICHWP6YkEj+D3WFU1HqxoWEAAAAAElFTkSuQmCC" /></span></strong></p>
    <p style="margin-bottom: 5.0pt;"><strong><span style="font-size: 8.0pt; font-family: 'Arial',sans-serif; color: #5c881a;">&nbsp; &nbsp; &nbsp; &nbsp; <img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAACYAAAAmCAYAAACoPemuAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAADsQAAA7EAZUrDhsAAAn0SURBVFhHtZh5XFTlGsdPpbEKIso2M4CGLWp2u3263W4WlqGWJoYMzDArmKR+ckkrA5cBdRhmzpxhc4MMTNEUt1xTs9zzZuZyhcp9lwwXVEBk+93nPZw+CCLk9v185g/e83Le533f5/k9z3O4B+AxTUaIx0hbN7+hKfIQvVnWU2/u3NOY3qVrnFUREGMJ9FIWKJ+Q5j56dKlBzxnsgRpjuixTbfPaEjnd9dTgJKfr4VOcboWbnKvCk5zKIqa6nVNZPHcb0/1y4tKCh+nsQS8qC7hHY+QHqT7/jnUECjp7x8IhZhdE826IywjA6OyeSJzXG8kLB9BvICZ+1Qdjc/6JoZkKqO3uiLS4IIbvcNLokM3Vpyj6Sq97cJS2ID+DIDcb0jqdiOZdQaeApPwBWLrNhl+ObMTxCwdw8eppXC27iNKyP1FSehan/jiEg8e3YM2PM5C6JIqMDKKNuELj8L5o4AO+0FiDn5Fef3/o7IpexnT/bWreA3rBF45lBuw7+p1owN+l7OZV/HZ6N7LXjcXQjECoeHcY0vwO6fnASGmZeyNOCBxicPicjra54tO5r2FX4QrcqqqQlrt3qmuqcOjENiTnD0K0zR0a3vuG0aYYZzJxj0tLto5RCFTqhI4lUVY32Jfqxat6WLATzN0wAerU9qDgqdVbFRNoycfqV24Bg0XWWyv4FEelumH2mtEov3lNeuXDo6a2Bst3CFBbvVhg3NTaFEZp+eaJtvgGGwT/n1U2N6SvGIryyodv1F/U1tVi0Q/ToLK1g473uaBKlr0imXEnBpssS21vh8S8t+7Jwe8X5rMCuYra7kYBIVsdbgpqL5nSgMYc0Edj63g5Nl0mhvr9UltXg4rK66IvVVaVS6P1VNVUoq6uTvqrnvOXjuKjWT3J3zxrDPagxlcan821NQqyvCjehcL6Y9SSD9zOjYor2FW0HDdvlUkjzXP8wj7MWfsxRs98GcPTu2PSvHew/qc5+JOCZ8/v6yAsG4pjNKcpq0nv1CQjRsF/y3sJPr6SWRwXlSx/WZ3qVTwsqzOOnN0rTW/gmx8zMNjUDvmbk1BVXSmNNmbfsU2ItT+NNz/hoE7xx1ChKwZNdkW/zzmMzOoJbaoc7050wsLvk1F85bho4OmLhaghGbl07TzG57yCKIt7NUlIuGQWx8XagsZGWpxpRwY6/sZaxXzt05zX0S+Bw8DJbUWhZOp+O+xERs18EWETOHo+BkWnd9KiRdhdtBLmRZEYNOVJRE71JOP8obXIMczRHab5A7D1wNe00Zug+xUDIcrmAmOabEZoKNeGCzUFOevsvss0gic2/zJfWqqBwlM7QFpDJxCCD9O7oc9nHD7J7oXv9uXhevklcc4P+/PFk7ItUeNWdeONMV/L/95Ep+iD95NcyTgZZq8eg8KT2xv54IHj31PulSPa4nVwBFUmnNbUxSfS7HFk+MyuOHxmjzStgRPFBxAnPIWPZryIX0/vAl+gFU9vwKS2GDf7VeRunIDJ8/qjf8Lj2Hbwa+m/GrOzcBkiktwxPKMHbXS7NNqYP66exGdf9sLgJJebcSyXfmjt0jU8ybn889zQZhW+sqoMSfMH4j26xuU77OLVbj24CIm5b4vX827iE+IpxvLB5J93boxFYebKYXgn8TGs35Mtjd5JZXU5+KUaRExzplNVhHFxKQGvUj0F8+IhdOyl0rQGThYfxEdZL5B/tYHK3Ak/kt8w2JUVntyBgq0WZNDC7DqZojellKqOcdmvItrsjXMlh6XR5pm7fjwiU1yht/loOa3dPyxiKjn+cgMtRo7YhGPn9yGO70KGPUEGxFNEnZCe/D1O0MaMfGeMyHxe9LeWyN88BUqLG2JSfIZzVNaIhqUtNzYrBTW11cheO5augkPK11FiBN4Lu4pW4L1Jzpi2MIIqjFvSaPOwIIkiw9TmTiM4gznwtcHJTrAsVt41N166dg6f5PQif+IwZva/sOfwWulJy7BSJ+ubD8WIXfffWTTSWPWbkrvhUyjpKnW8n4EMC3qWavXKxHlv3aFPt5Ox8gP0JbGs97MV0mjLsPorIrk9CW9Iqy7AbstB7sSc32CT9+dU5h6+kdPdT42c/ZzoT3dj+/8Wkw5R9WkLxFraffHl43TCpeILm+Y/BjvlhC/DEDqew/LtvDR6d0pKzyAhrzdlGOeqWD6kGzfQ5O+q4Tut0zq8aPECaVrzrNyVhujp3iQT7Ui9nyXJ6IeNe+eiuolvXrlRDFuBBm+M40SFv3rjovTk7hSd2olhWcGIMnv+rjY93VFMSUY+KEFpdcLM1SPFCqApv5/9iU5pJuZtShTFlmnaoCnOSPpqEPUAm6RZ9TBFZ4LLTmrsrJdxtuQ36UnLrNzpEBsWg8M/Nz7+pbb1hlkVr0enelwZOfNZynG/SlPruUZpJzG3L8JIRFlKibH4InlBuBgALGL/mrP3yLeYQxVvjMUPvcnZJ+b1w5k/i8TnrcG0LoFqQKWZDBOCVKJRDO14Xzfq+wqYxfmbTdL0ei5dP08yoaRTelL8Kad5kaFhyFk3DjNWjaAkPYTKnJfELPA2JfEYiz8WUNhfKy+R3tA6m/cvQAzlaj3v+3PExOAgyax6dBZFeIzVqyI+8ykKgv3Sv7CUUisWflsOLBR1bHhmDxJAP7EMGjTFjaKOSuPUAHz2RahYFrGqoq4VWbidK9eLqQv7D7V1HrU6qnIkcxoYNYpz0vMBC1XUPU9fOISMuSH9awMs+i5Qxbmfesvth5ZSsCzBz4fXkzGHms0arcEK0i8oDamoqzemBWxVmRQBkjmNiTLLe+jtvkdV1PflbUygIq7ehx4V3+7JYV0S/Tpd06aQdrWEziZTau3eN1RWTyzZYnlkxm05sAg6ewDV+u3rpN6ydWjiGGpGq5hxeRs+p97yzqrjfmH5cvXuLJZ26KQ8YODl6f1HhThJS7cMa9uNdsUYjd27jPV90xdFkJbtEXvBB+F8yVExd8bwrNFtX2UQ5EI8Cby07N9HZ1VEG9J9f2UBEZ8VgvmbJlF9dm+OXktad+HSMazYmYbRs18QP6roBZ8L1MOOfekvIb0ftBQQBkfAAo3QoVRFOjeCRHgG7XrbwcU4cm4v5bhzqKC2rqa2ShRcVsdfJglgn6d2F60Si78xc/4hfitT2TyrjBmyNTpBHiq9/sEINXFtYu2BA+PSZYtUqe0vK1Op/KXcOnLWc0ic1wfWAhUyV32ArFXxsFN5PHl+P4ya8zz0aZ0QZXWh/NeuIi5TtjpWkGk1pg4e0msfHmGUIdg3s7gMxQS90PGbSLPbscFJzhWsnmPlCvu9n+wM9slzyHS3MzG890baTHKso/PbkYK8g/SaRwst5DKc7+ITlxb8jC5V0UtvU/TV2eX99Lz8Db3jqe7so7HS1M1dmn6PcNz/AYwf2egaJIx/AAAAAElFTkSuQmCC" /><img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAACYAAAAmCAYAAACoPemuAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAADsQAAA7EAZUrDhsAAAiBSURBVFhHtZh5VFT3FcfHpIZFEJB95g0DFo3HNkkrnh6T1tKT5tQlKqgzzAyzD4pL61Gsx5CoDMts782bYVVxQQXUKhpNSaqoGKM1mkQao9VKtBQVF3ADFSIicHvf4xfRioUB/XDm8Mf7Ld/3u7+7PcEAGKTKjR46nxkdpmXEP9XbqDe0lqg3NWzUSK15hGieLSIg1hT7EzL25aPKjHzd6JQokrKpnERm2MGZ5iG18eke9+LSPB7GmTzb4tI9mmdkeV9V2Py+1rnCivTOyDm6zIiY5OSYwWSJF4uGFo81uCLsamfwqZkWL1Aw3mDIDYcFhW/CR5t+Bxlb3sffFFhe8h6krIuBWXkRkOj0BanNE/AFLhtc1EYdQ00EEAwiSw4MqSU6WOcQpeuyQ87LmSGgyw4FU8kkKPvCBlXnK6Dm2nfQ0HgJGpsboKn5BtxsqoNLDWfhVM0h+Oyr1UCXKWB2XhTIHV6gdgXe1LLUJq1L/DOyfP/Q2qN+pc8Or1Q6/EDjCgKmTAVV31dA4/0G6CvND5rg+7pvoGjvEhQYCQp2COizQ8/pGSqRbOMeWlo8TecMq5E7vGHx2nFw+PR2aH3YQrZzn/aOR/CvS8fAvHUGXgMfUDmGtehYSWqsSdB3B9Ez4ji1M+R6Au0N9HYlXLtVQ5YfOC2td6H0wApIpIeBkvYHnYNKk8kEr5Ktn486S/iOhg2pS7B7Qd4nc+D+D7fJki+Ozs5OKD+Wj04RyDlGm8Yqnku27xnVUhGlc4Z/yZnPUaZGUY1kqZfDjiMMKBk/UDuCb6gs4vFExrOoGRGbyPrC0vXj4dbdq2R6/3jwsBluNF2G67f/wzvLo/Y28qSbtvZWtEpyl0M4hfukH4YFEyndcIrVbFA9BkU4UV1BprrP+atVsPZvKZBa9C4sKBgDf8z/BaQUjoO04slQ+NkiPpQ8ScOdi7CwMAbDydAOzCDziJwuYpIFg/VOajUXawr+Oq/Ht+uNTvz7+z93gIGNgqkrBsOMDB+QZflDQlYAyMwBMDXtNZBlBsG3F/aTGd1UfLMOlGgpHRt2bDpeJyILTWgTvaWkAy8b8yg4e/EYGe4edTerYX7eWzBl+SugsoWDxi7C/2G8wGkoasKHApiTPYoPwP/LnXv18EHReJBZOZNGyYgsDA+Yz7jUQW+Xww8P75Ph7lH5bQkvQmkNfixKYQ6ChavGQubmOFi06h3I/tgIHZ3tZMbT7DziQHN6g9YlXJ+MFhTITKNf0zhCN3NHuefrtWSY+2w7ZIY4kweo7eGgxhObmeEL1r/IoO7GOfTuO3za4pzheZytPQqz8yNBbvWr1lhHBQqSnKOHJVj8Ts/Oj4IzF4+SYe5TfGAZxKHJ1HYhKCzBIM0cii9aSJ72zs27V7AYeBdfzrNdb4t6Q2AsiJTEZ3g1Lln/Nly7fYEMc58tn2fwd4kTJrcEQYJ5GJq3mDztnTZ0uJzdRphh9sS4JpwqMLKSX8aleXZwZcvdlptkWN+4VH8GPj1egFXESv4ecV7ICUu0hYLCGgKunXrYV1UE5cfzYe+Jtbj+LTKzZ4r3fwQymzdomPDZAj0tHj893QMrBwVe/GYypG9UVK2HycsG8aFAjhedu/ScsK6fCE8tAOLTveH95a+CEk+x9vopMrNntn9hBTmNCZ4JWiTQOqjf8sJ2KPlo7Q6VJ4sxNvnjnQp6QtCTv3DeO7lYlsQOx+B6hszsmbLDRJgdhWlc4rHoTZ2ZW6fBvRb3EnYFmmnSMk8MqNyJBYKG7j4xDS1E0z55YsG9nlhpZRok2IeAxhoyV2BwjRgen+55b2nRb/i85g41107CNqxkyw7bsKqd3H3HrKEYz0LAvk0Ju466eBPtxv9NzfVk5rNw9Vo+5k2ubNcy1HS+dJZafKrnFkRD9eWvyDD32XooE71yMC+syysDYf8/NpKnvXMHE/2KkglcuOjU0xFjBBNzoz00zpAdKqcfH737S8mB5RjHuADLxbGucFFRtY487Z1qLL/nFIwAqdm3NskSFcqnJINDnCK1e0DOLiM8fPSADHWPrgD7tDAuRPSVT4+vBIXDB7C62cplI16YyhwyTmH3vzEnPxr+jV1PfxiIMM7puM5LijFMx4r1vCiOiQuiPfTZVImC9YKN+1L50tddBiLsyKkyULEBgDn7tNRERRNZXSSaRZMSmYB7SbkSOHfJfSforzCuF03dEIudk2+njpGkEjndcJ0KxqENStYH0kun8P2gO/RHGFdcbtq3DJR4t7B/PZ6YFSYhcp5GbhWO1LrCzyqw4163ZzE6QitZonf6I4yLAtiIgJoNvK+xhccRGT2jdoinqtigRjntC5sPpkNbH8W5K+zLMzuxWqW4LqkTA6qJbP//4fo87PceyO2+2Fgs7lNv2Vdh7R3tWKetQe8ToQmHgsElXil1Ul5k614ZpGUk8/DkmrgPKRmbp2EvcBQ6cNHn0Rdh9XdqYVX5n0DlwEbXEdCOovKN9Ou+ZM++g+Km63LCTivQIbgPIhsqUvmvO61tz37DKD1oelwo/ijs4MlS/hmXg8uxblu4ZgzfQ2qzg+u1DvEHsSaJJ9nKfZQWySgsjTaqXYG3uWaBSxu5u5Pg0Hdb4HzdCWyMr/DlUknliscnxiVwLpGvLl8AWz/PgpQ1MdieYaNC+7UbskV7NUzEe2T5AWISvKJ1iCZhEC5V0P4NMjuWv64AmL9yFCwr/j3Wcgr485q3QWnDDokR8vfH4KJAlxMKCYw3yCw+rcZc4R6tnTLEmST+ZNUXhw6PfpZTMs7oiliiYYN3YVVyAYvMlvgMD1DaA9HLxKBlsR6jhW0ah/CK1hlaqc8RmfVs5ASDpYf2/2UgTaG81I7hIbPYqJE6S8SvtVbxH9QMNVFJh8WqLdTP5zPBYUY60P2LzSMQ/BcN9rWAK9SSrQAAAABJRU5ErkJggg==" /></span></strong></p>
    <p style="margin-left: 2.55pt;"><strong><span style="font-size: 22.0pt; font-family: 'Arial',sans-serif; color: #4f8a10;">&nbsp;&nbsp; &nbsp;</span></strong><strong><span style="font-size: 22.0pt; font-family: Webdings; color: #5c881a;">P</span></strong><strong><span style="font-size: 7.5pt; font-family: 'Tahoma',sans-serif; color: #5c881a;">&nbsp; </span></strong><strong><span style="font-size: 6.0pt; font-family: 'Arial',sans-serif; color: #5c881a;">PRESERVE O MEIO AMBIENTE. S&Oacute; IMPRIMA ESTE E-MAIL SE FOR INDISPENS&Aacute;VEL.</span></strong></p>
    <p style="margin-left: 2.55pt;"><strong><span style="font-size: 6.0pt; font-family: 'Arial',sans-serif; color: #5c881a;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </span></strong><strong><span style="font-size: 6.0pt; font-family: 'Arial',sans-serif; color: #5c881a;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span></strong><strong><span style="font-size: 6.0pt; font-family: 'Arial',sans-serif; color: #5c881a;">PROTECT THE ENVIRONMENT. PRINT THIS E-MAIL ONLY IF EXTREMELY NECESSARY.</span></strong></p>
    <p style="margin-left: 2.55pt;"><strong><span style="font-size: 6.0pt; font-family: 'Arial',sans-serif; color: #5c881a;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </span></strong><strong><span style="font-size: 6.0pt; font-family: 'Arial',sans-serif; color: #5c881a;">PROTEJA EL MEDIO AMBIENTE. IMPRIMA ESTE MENSAJE S&Oacute;LO EN CASO NECESARIO.</span></strong></p>


    """

    return text

print("\nGerando email...")

os.startfile("outlook")

outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)

mail.Subject = "Acompanhamento Diário dos Serviços de Recuperação de Energia"
mail.HtmlBody = body()
mail.To = ''

print("Sucesso na criação do email!")
mail.Display(True)




Gerando email...
Sucesso na criação do email!
